In [7]:
import gensim
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd 
import json

IMPORTING DATA 

In [8]:
data = pd.read_csv("../input/twitter-data-fromapi/label3.csv", index_col = 0)
# print(data.head())
data['hate_speech'] = data['hate_speech'].replace({1: 'hate', 0: 'not hate'})
# emotions = ['Happy', 'Angry', 'Surprise', 'Sad', 'Fear']
max_emotions = []
# iterate over each row in the DataFrame
for index, row in data.iterrows():
    emotion_scores_string = row['emotion'].replace("'", "\"")
    # load the string as a dictionary
    emotion_scores_dict = json.loads(emotion_scores_string)
    # get the emotion with the highest score for the current row
    max_emotion = max(emotion_scores_dict, key=emotion_scores_dict.get)
    # append the max emotion to the list
    max_emotions.append(max_emotion)

data["emotions"] = max_emotions
data = data[["clean_tweets","polarity","emotions","hate_speech"]]
df = data
data.head(10)

,clean_tweets,polarity,emotions,hate_speech
0,ukrainian president zelensky visited ukrainian...,Neutral,Surprise,not hate
1,youtubewatch v wbessn despicable tank take cov...,Negative,Fear,not hate
2,drawing process update latest portrait ukraine...,Positive,Fear,not hate
3,militant special force unit stepan kurilo liqu...,Positive,Fear,not hate
4,much changed since feb 2022 anyone remember de...,Neutral,Happy,not hate
5,u fighting hegemonic war russia china think pi...,Negative,Sad,hate
6,rusia ukraina listen dudayevvladimir putin thi...,Negative,Surprise,hate
7,ustops sharing data nuclear force russia wsj s...,Neutral,Fear,not hate
8,nazi state disliking fearing russia make ukrai...,Neutral,Sad,hate
9,theguardiannews 2015 feb west european america...,Positive,Fear,not hate


#  Preprocess the text data

In [9]:
def preprocess_text(text):
    tokens = gensim.utils.simple_preprocess(text)
    return tokens

# Apply the preprocessing function to the 'tweets' column
data['tokens'] = data['clean_tweets'].apply(preprocess_text)

# Create a list of tuples for preprocessed data
preprocessed_data = [(tokens, labels1, labels2, labels3) 
                     for tokens, labels1, labels2, labels3 in zip(data['tokens'], data['polarity'], data['emotions'], data['hate_speech'])]

In [10]:
# Preprocess the text data
# Train the Word2Vec model
model = gensim.models.Word2Vec([tokens for tokens, _, _,_ in preprocessed_data], min_count=1)

# Extract features for each text sample
X = []
y_emotion = []
y_sentiment = []
y_hate = []
for tokens, emotion, sentiment, hate in preprocessed_data:
    vector = np.mean([model.wv[token] for token in tokens], axis=0)
    X.append(vector)
    y_emotion.append(emotion)
    y_sentiment.append(sentiment)
    y_hate.append(hate)

# Train a classifier for emotion classification

In [11]:
clf_emotion = LogisticRegression()
clf_emotion.fit(X, y_emotion)

# Train a classifier for sentiment classification
clf_sentiment = LogisticRegression()
clf_sentiment.fit(X, y_sentiment)


clf_hate = LogisticRegression()
clf_hate.fit(X, y_hate)

# Evaluate the model
y_pred_emotion = clf_emotion.predict(X)
y_pred_sentiment = clf_sentiment.predict(X)
y_pred_hate = clf_hate.predict(X)

print("Emotion classification report:\n", classification_report(y_emotion, y_pred_emotion))
print("Sentiment classification report:\n", classification_report(y_sentiment, y_pred_sentiment))
print("Hate classification report:\n", classification_report(y_hate, y_pred_hate))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Emotion classification report:
               precision    recall  f1-score   support

    Negative       0.79      0.13      0.23      2156
     Neutral       0.47      0.38      0.42      4035
    Positive       0.46      0.74      0.57      4140

    accuracy                           0.47     10331
   macro avg       0.57      0.42      0.40     10331
weighted avg       0.53      0.47      0.44     10331

Sentiment classification report:
               precision    recall  f1-score   support

       Angry       0.00      0.00      0.00       520
        Fear       0.40      0.85      0.55      3877
       Happy       0.39      0.28      0.33      3025
         Sad       0.00      0.00      0.00      1589
    Surprise       0.00      0.00      0.00      1320

    accuracy                           0.40     10331
   macro avg       0.16      0.23      0.18     10331
weighted avg       0.27      0.40      0.30     10331



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Hate classification report:
               precision    recall  f1-score   support

        hate       0.73      0.28      0.40      2019
    not hate       0.85      0.97      0.91      8312

    accuracy                           0.84     10331
   macro avg       0.79      0.63      0.65     10331
weighted avg       0.82      0.84      0.81     10331



In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["polarity"] = le.fit_transform(df["polarity"])
df["emotions"] = le.fit_transform(df["emotions"])
df["hate_speech"] = le.fit_transform(df["hate_speech"])